Prerequisite: psycog2-binary, sqlalchemy, and ipython-sql libraries

pip install psycog2-binary sqlalchemy ipython-sql

In [1]:
# load ipython-sql extension
%load_ext sql

In [2]:
# connect to database
%sql postgresql://postgres@localhost:5432/paintings

In [3]:
#import python libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

### Analysis

Q1: which countries have the highest number of artists in the `artists` table?

In [4]:
%%sql
SELECT 
    COUNT(artist) AS artist_count,
    nationality,
    RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
FROM artist
GROUP BY nationality
limit 5;


 * postgresql://postgres@localhost:5432/paintings
5 rows affected.


artist_count,nationality,rank
115,French,1
83,American,2
68,Dutch,3
44,English,4
26,Italian,5


Q2: what are the popularist styles of paintings that can be found in the museum collections? 

In [5]:
%%sql
select count(artist_id) as style_count, style, rank() over(order by count(*) desc) as rank 
from artist
group by style
limit 5;

 * postgresql://postgres@localhost:5432/paintings
5 rows affected.


style_count,style,rank
54,Baroque,1
42,Impressionist,2
35,Realist,3
25,Expressionist,4
23,Rococo,5


Q3: which aritists had the longest lifespan? 

In [6]:
%%sql
select full_name, (death-birth) as lifespan, rank() over(order by death-birth desc) as rank
from artist
limit 10;

 * postgresql://postgres@localhost:5432/paintings
10 rows affected.


full_name,lifespan,rank
Sir George Clausen,92,1
Kees Van Dongen,91,2
Titian,91,2
Asher Brown Durand,90,4
Buonarroti Michelangelo,89,5
Frank Weston Benson,89,5
Thomas Sully,89,5
Christian Rohlfs,89,5
Katsushika Hokusai,89,5
James Ensor,89,5


Q4: whose artwork appear in the most collections?

In [ ]:
%%sql
select a.full_name, a.nationality, count(b.artist_id) as count_artworks, rank() over(order by count(*) desc) --array_agg(b.name) as artworks
from artist a
join work b 
on a.artist_id=b.artist_id
group by a.full_name, a.nationality
limit 5;

 * postgresql://postgres@localhost:5432/paintings
5 rows affected.


full_name,nationality,count_artworks,rank
Pierre-Auguste Renoir,French,469,1
Claude Monet,French,378,2
Vincent Van Gogh,Dutch,308,3
Maurice Utrillo,French,253,4
Albert Marquet,French,233,5


Q5: Which styles of artwork are popular in the collections?

In [33]:
%%sql
select style, count(*) as style_count, rank() over(order by count(*) desc)
from work
group by style;

 * postgresql://postgres@localhost:5432/paintings
24 rows affected.


style,style_count,rank
Impressionism,3078,1
Post-Impressionism,1672,2
None,1228,3
Realism,1179,4
Baroque,972,5
Expressionism,673,6
Fauvism,653,7
Rococo,535,8
Romanticism,511,9
American Art,461,10


Q6: which first name is common among artists?

In [6]:
%%sql
select first_name, count(first_name), rank() over(order by count(*) desc)
from artist
group by first_name
limit 10;

 * postgresql://postgres@localhost:5432/paintings
10 rows affected.


first_name,count,rank
John,24,1
Jean,22,2
Jan,14,3
Charles,11,4
George,11,4
William,11,4
Thomas,10,7
Paul,8,8
Henri,7,9
Georges,6,10


Q7: which 5 aritists have the most artworks in each museum?

In [9]:
%%sql result <<
WITH count_collections AS (
    SELECT 
        a.full_name, 
        c.name AS museum, 
        COUNT(*) OVER(PARTITION BY c.name, a.full_name) AS total_collection
    FROM artist a
    JOIN work b ON a.artist_id = b.artist_id
    JOIN museum c ON b.museum_id = c.museum_id
)
SELECT full_name, museum, total_collection
FROM (
    SELECT 
        full_name, 
        museum, 
        total_collection, 
        ROW_NUMBER() OVER (PARTITION BY museum ORDER BY total_collection DESC) AS row_num
    FROM (
        SELECT DISTINCT *
        FROM count_collections
    ) sub
) ranked
WHERE row_num <= 5;


 * postgresql://postgres@localhost:5432/paintings
150 rows affected.
Returning data to local variable result


In [11]:
top5_artist_by_museum = pd.DataFrame(result)
top5_artist_by_museum.head(n=10)

,full_name,museum,total_collection
0,Félix Vallotton,Army Museum,1
1,Frederick Carl Frieseke,Chrysler Museum of Art,1
2,George Inness,Cleveland Museum Of Art,10
3,Odilon Redon,Cleveland Museum Of Art,5
4,Francesco Guardi,Cleveland Museum Of Art,4
5,David Teniers,Cleveland Museum Of Art,4
6,Pierre-Auguste Renoir,Cleveland Museum Of Art,4
7,Claude Monet,Columbus Museum of Art,1
8,Vincent Van Gogh,Courtauld Gallery,1
9,Félix Vallotton,Dallas Museum of Art,1


Q8: check the opening hours of a museum

In [ ]:
%%sql
--drop function check_opening_days(text);

create or replace function check_opening_days(museum_name text)
returns table (
    museum text,
    day text,
    open time,
    close time,
    opening_hours text
) as $$
begin
    return query
    select a.name, b.day, b.open, b.close, to_char((b.close - b.open), 'HH24:MI') as opening_hours
    from museum a
    join museum_hours b on a.museum_id = b.museum_id
    where a.name = museum_name;
end;
$$ language plpgsql;

 * postgresql://postgres@localhost:5432/paintings
Done.
Done.


""


In [104]:
%sql SELECT * from check_opening_days('Chrysler Museum of Art');


 * postgresql://postgres@localhost:5432/paintings
6 rows affected.


,museum,day,open,close,opening_hours
0,Chrysler Museum of Art,Sunday,12:00:00,17:00:00,05:00
1,Chrysler Museum of Art,Tuesday,10:00:00,17:00:00,07:00
2,Chrysler Museum of Art,Wednesday,10:00:00,17:00:00,07:00
3,Chrysler Museum of Art,Thusday,10:00:00,17:00:00,07:00
4,Chrysler Museum of Art,Friday,10:00:00,17:00:00,07:00
5,Chrysler Museum of Art,Saturday,10:00:00,17:00:00,07:00


Q9: check the museum where an artwork is stored

In [ ]:
%%sql

--drop type address_type;

create type address_type as (
    address text,
    city text,
    country text
); 


--drop function collection_museum(text);

create or replace function collection_museum(artwork text)
returns table (
    name text,
    style text,
    artist text,
    museum text,
    address address_type,
    url text
) AS $$
BEGIN
    RETURN QUERY
    SELECT DISTINCT 
        b.name, 
        b.style, 
        a.full_name, 
        c.name, 
        row(c.address, c.city, c.country)::address_type, 
        c.url
    FROM artist a
    JOIN work b on a.artist_id=b.artist_id
    JOIN museum c on b.museum_id=c.museum_id
    WHERE b.name=artwork;
END;
$$ LANGUAGE plpgsql;

 * postgresql://postgres@localhost:5432/paintings
Done.


""


In [230]:
%sql select * from collection_museum('Dante and Virgil in Hell')

 * postgresql://postgres@localhost:5432/paintings
1 rows affected.


,name,style,artist,museum,address,url
0,Dante and Virgil in Hell,Classicism,William Adolphe Bouguereau,Musée d'Orsay,"(""1 Rue de la Légion d'Honneur"",Paris,France)",https://www.musee-orsay.fr/en


Q10:check the infomation, such as style, subject, product size, and price of an artwork

In [12]:
%%sql 
CREATE EXTENSION IF NOT EXISTS plpython3u;

CREATE OR REPLACE FUNCTION check_artwork_info(artwork TEXT)
RETURNS table ( 
    name text,
    subject text,
    size text,
    sale_price integer,
    regular_price integer,
    url text
) 
AS $$
import pandas as pd
sql = """
SELECT DISTINCT a.name as name, b.subject, 
  case 
    when d.label is not null
    then d.label
    else c.invalid_size_id
  end as size,
  c.sale_price,
  c.regular_price,
  e.url
FROM work a
JOIN subject b ON a.work_id=b.work_id
JOIN product_size c ON c.work_id=a.work_id
LEFT JOIN canvas_size d ON d.size_id=c.size_id
JOIN image_link e ON e.work_id=a.work_id
WHERE a.name=""" + plpy.quote_literal(artwork)
    
result = plpy.execute(sql)
data = [dict(row) for row in result]
df = pd.DataFrame(data) 
if df.shape[0] == 0:
  return df.to_dict('records')
else:
  colnames = df.columns.to_list()
  colnames.remove('subject')
  dup_idx = df.loc[:,colnames].duplicated()
  clean_df = df[~dup_idx].reset_index(drop=True)
  dup_df = df[dup_idx].reset_index(drop=True)
  clean_df['subject'] = clean_df['subject'] + '/' + dup_df['subject']

  return clean_df.to_dict('records')
$$ LANGUAGE plpython3u;




 * postgresql://postgres@localhost:5432/paintings
Done.
Done.


[]

In [13]:
%sql result << select * from check_artwork_info('Mount Adams, Washington')

 * postgresql://postgres@localhost:5432/paintings
5 rows affected.
Returning data to local variable result


In [14]:
print(result)

+-------------------------+---------+----------------------------+------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|           name          | subject |            size            | sale_price | regular_price |                                                                                                                                         url                                                                                                                                         |
+-------------------------+---------+----------------------------+------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------